# 機械学習モデルによる画像特徴量抽出と近似最近傍探索を用いた類似画像検索 

## References
#### Basics  
- [Similar Images Recommendations using FastAi and Annoy](https://towardsdatascience.com/similar-images-recommendations-using-fastai-and-annoy-16d6ceb3b809)
- [Content Based Image Retrieval Using a Convolutional Denoising Autoencoder](https://medium.com/sicara/keras-tutorial-content-based-image-retrieval-convolutional-denoising-autoencoder-dc91450cc511)  

#### Advanced
- [Baseにおける類似画像検索](https://logmi.jp/tech/articles/322876)
- [Yahooにおける類似画像検索](https://techblog.yahoo.co.jp/entry/2020081130014621/)

## Overview  
- There are two main xxx, image feature extraction using pretrained deep-learning model and approximate nearest neighbors.
- Use two pretrained deep-learning model, '[DeeplabV3](https://github.com/tensorflow/models/tree/master/research/deeplab)' and '[EfficientNet](https://arxiv.org/pdf/1905.11946.pdf)' to extract image feature.
- Use python package by Spotify, named '[annoy](https://github.com/spotify/annoy)' for approximate nearest neighbors.

# 1. Indexing

##### Import modules

In [ ]:
import os
import sys
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub
from annoy import AnnoyIndex
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath('..'))
from libs.deeplabv3.model import Deeplabv3

### 1.1. Extract features (vector embeddings)  from images using pretrained ML model

##### Define feature-extractor

In [ ]:
class FeatureExtractor(tf.keras.Model):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.deeplab = Deeplabv3()
        self.deeplab_person_region_index = 15
        efficientnet_intermediate_layer_url = "https://tfhub.dev/google/efficientnet/b0/feature-vector/1"
        self.efficientnet_input_shape = [224,224,3]
        self.efficientnet_intermediate_layer = hub.KerasLayer(
            efficientnet_intermediate_layer_url,
            input_shape=self.efficientnet_input_shape)

    def call(self, images, training=False):
        deeplab_outputs = self.deeplab(images)
        person_region_masks = tf.cast(tf.math.argmax(deeplab_outputs, -1) == self.deeplab_person_region_index, tf.float32)
        masked_images = images * tf.stack([person_region_masks, person_region_masks, person_region_masks], axis=3)
        resized_and_masked_images = tf.image.resize(masked_images, self.efficientnet_input_shape[:2])
        self.image_features = self.efficientnet_intermediate_layer(resized_and_masked_images)
        return self.image_features
    

##### Create input pipeline 

In [ ]:
def load_and_preprocess_image(image_path):
    '''preprocess image to feed feature-extractor with appropriate input'''
    target_image_shape=[512, 512]
    num_channels = 3
    raw_image = tf.io.read_file(image_path)
    image_tensor = tf.image.decode_jpeg(raw_image, channels=num_channels)
    resized_image_tensor = tf.image.resize(image_tensor, target_image_shape)
    resized_and_normalized_image_tensor = resized_image_tensor/255.0
    return resized_and_normalized_image_tensor

AUTOTUNE = tf.data.experimental.AUTOTUNE
image_dir = "/Volumes/Samsung_T5/19MH228/similar-image-retrieval/images/images_season_id_22"
image_path_list = glob.glob(image_dir + '/*.jpg')[::100]
num_images = len(image_path_list)
image_ids = np.array([int(os.path.splitext(os.path.basename(image_path))[0]) for image_path in image_path_list])
batch_size = 1
image_path_dataset = tf.data.Dataset.from_tensor_slices(image_path_list)
image_dataset = image_path_dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE).batch(batch_size)

##### Extract image features from images

In [ ]:
feature_extractor = FeatureExtractor()
image_features = feature_extractor.predict(image_dataset, verbose=1)

##### Save ids and features

In [ ]:
feature_save_dir = '../extracted-features'
if not os.path.isdir(feature_save_dir):
    os.mkdir(feature_save_dir)
image_ids_save_path = '../extracted-features/image-ids.npy'
image_features_save_path = '../extracted-features/image-features.npy'
np.save(image_ids_save_path, image_ids)
np.save(image_features_save_path, image_features)

### 1.2. Build index

##### Build annoy model

In [ ]:
image_ids_save_path = '../extracted-features/image-ids.npy'
image_features_save_path = '../extracted-features/image-features.npy'
image_ids = np.load(image_ids_save_path)
image_features = np.load(image_features_save_path)
image_vector_dims = image_features.shape[1]
num_trees = 50
metric = 'euclidean'
annoy_model = AnnoyIndex(image_vector_dims, metric)
for image_id, image_vector in tqdm(zip(image_ids, image_features)):
    annoy_model.add_item(image_id, image_vector)
annoy_model.build(num_trees)

##### Save index model

In [ ]:
model_save_dir = '../models'
if not os.path.isdir(model_save_dir):
    os.mkdir(model_save_dir)
annoy_model_save_path = "../models/{}-{}-{}.ann".format(image_vector_dims, metric, num_trees)
annoy_model.save(annoy_model_save_path)

<br>

# 2. Retrieval

### Evaluate 

##### Check avairable annoy models

In [ ]:
model_save_dir = '../models'
print('Avairable annoy models -> {}'.format(glob.glob(model_save_dir+'/*.ann')))

##### Test similar image retrieval

In [ ]:
annoy_model_path = '../models/1280-euclidean-50.ann'
metric = os.path.basename(annoy_model_path).split('-')[1]
num_feature_dims = int(os.path.basename(annoy_model_path).split('-')[0])
annoy_model = AnnoyIndex(num_feature_dims, metric)
annoy_model.load(annoy_model_path)
ext = ".jpg"
image_id_and_arrays = []
for query_image_path in tqdm(glob.glob(image_dir + "/*"+ext)[::900]):
    num_neighbors = 5
    query_image_id = int(os.path.splitext(os.path.basename(query_image_path))[0])
    nearest_neigbor_ids, distances = annoy_model.get_nns_by_item(
        query_image_id, 
        num_neighbors, 
        search_k=-1,
        include_distances=True
    )
    query_image = Image.open(query_image_path)
    query_image_array = np.array(query_image)
    fig = plt.figure(figsize=(40, 20))
    ax = fig.add_subplot(1, num_neighbors+1, 1)
    ax.imshow(query_image_array)
    ax.set_title('query\nimage_id -> {}\n'.format(query_image_id))
    ax.axis("off")
    for i, (neigbor_image_id, distance) in enumerate(zip(nearest_neigbor_ids, distances)):
        image_file = os.path.join(image_dir, str(neigbor_image_id)+ext)
        image = Image.open(image_file)
        image_array = np.array(image)
        ax = fig.add_subplot(1, num_neighbors+1, i+2)
        ax.imshow(image_array)
        rank_statement = "1st" if i == 0 else "2nd" if i == 1 else "3rd" if i == 2 else "{}th".format(i+1)
        ax.set_title('{}\nimage_id -> {}\ndistance -> {}'.format(rank_statement, neigbor_image_id, distance))
        ax.axis("off")
    plt.show()